In [ ]:
!pip install pyterrier
!pip install pyterrier[java]

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.1/208.1 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 866.1/866.1 kB 45.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 71.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 304.8/304.8 kB 26.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.0/149.0 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.6/45.6 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 59.4 MB/s eta 0:00:00
  Created wheel for warc3-wet-clueweb09: filename=warc3_wet_clueweb09-0.2.5-py3-none-any.whl size=18919 sha256=62cc5081c4e8f91b18686660a509950998c5b32bbf41aab85a28dc237952a8b3
  Stored in directory: /root/.cache/pip/wheels/f6/85/c2/9f0f621def52a1d5db7d29984f81e45f9fb6dfeb1a4eb6e31c
 

In [ ]:
import pyterrier as pt
import pickle
import pandas as pd
import shutil
import os
import re
from pathlib import Path
from pyterrier.measures import *

# Data exploration

In [ ]:
dataset = pt.get_dataset('irds:beir/fiqa/test')

In [ ]:
dataset.get_corpus_iter()

[INFO] [starting] building docstore
[INFO] [starting] opening zip file
[INFO] If you have a local copy of https://public.ukp.informatik.tu-darmstadt.de/thakur/BEIR/datasets/fiqa.zip, you can symlink it here to avoid downloading it again: /root/.ir_datasets/downloads/17918ed23cd04fb15047f73e6c3bd9d9
[INFO] [starting] https://public.ukp.informatik.tu-darmstadt.de/thakur/BEIR/datasets/fiqa.zip
docs_iter:   0%|                                     | 0/57638 [00:00<?, ?doc/s]
https://public.ukp.informatik.tu-darmstadt.de/thakur/BEIR/datasets/fiqa.zip: 0.0%| 0.00/17.9M [00:00<?, ?B/s]
https://public.ukp.informatik.tu-darmstadt.de/thakur/BEIR/datasets/fiqa.zip: 33.1%| 5.95M/17.9M [00:00<00:00, 15.3MB/s]
https://public.ukp.informatik.tu-darmstadt.de/thakur/BEIR/datasets/fiqa.zip: 66.3%| 11.9M/17.9M [00:00<00:00, 22.8MB/s]

[INFO] [finished] https://public.ukp.informatik.tu-darmstadt.de/thakur/BEIR/datasets/fiqa.zip: [00:01] [17.9MB] [13.3MB/s]
docs_iter:   0%|                                   

beir/fiqa/test documents:   0%|          | 0/57638 [00:00<?, ?it/s]

In [ ]:
queries = dataset.get_topics()
display(queries)

[INFO] [starting] opening zip file
[INFO] [finished] opening zip file s]
[INFO] [starting] opening zip file
[INFO] [finished] opening zip file s]


,qid,query
0,4641,Where should I park my rainy-day / emergency f...
1,5503,Tax considerations for selling a property belo...
2,7803,Can the Delta be used to calculate the option ...
3,7017,Basic Algorithmic Trading Strategy
4,10152,What does a high operating margin but a small ...
...,...,...
643,4102,How can I determine if my rate of return is “g...
644,3566,Where can I buy stocks if I only want to inves...
645,94,Using credit card points to pay for tax deduct...
646,2551,How to find cheaper alternatives to a traditio...


In [ ]:
qrels = dataset.get_qrels()
display(qrels)

,qid,docno,label,iteration
0,8,566392,1,0
1,8,65404,1,0
2,15,325273,1,0
3,18,88124,1,0
4,26,285255,1,0
...,...,...,...,...
1701,11039,330058,1,0
1702,11039,91183,1,0
1703,11054,155053,1,0
1704,11054,321015,1,0


In [ ]:
def ensure_java_home():
    if os.environ.get("JAVA_HOME"):
        return
    java = shutil.which("java.exe" if os.name == "nt" else "java")
    if java:
        jdk_parent = Path(java).resolve().parents[1]
        cand = jdk_parent / ("bin/java.exe" if os.name == "nt" else "bin/java")
        if cand.exists():
            os.environ["JAVA_HOME"] = str(jdk_parent)
            return
    if not os.environ.get("JAVA_HOME"):
        raise EnvironmentError(
            "JAVA_HOME is not set. Install a JDK and set JAVA_HOME to the JDK folder."
        )

ensure_java_home()

def indexing(index_path, ds):
    index_path = str(Path(f"./{index_path}").resolve())
    if os.path.exists(index_path):
        shutil.rmtree(index_path)
    indexer = pt.index.IterDictIndexer(index_path, meta={'docno':35})
    indexref = indexer.index(ds.get_corpus_iter())
    index = pt.IndexFactory.of(indexref)
    stats = index.getCollectionStatistics()
    return indexref, stats

def rmv_problems(text: str) -> str:
    text = str(text)
    text = text.replace(":", " ")
    text = text.replace("'", " ")
    text = text.replace('"', " ")
    text = re.sub(r"\s+", " ", text).strip()

    return text



In [ ]:
indexref, stats = indexing('beir-index', dataset)

terrier-assemblies 5.11 jar-with-dependencies not found, downloading to /root/.pyterrier...


https://repo1.maven.org/maven2/org/terrier/terrier-assemblies/5.11/terrier-assemblies-5.11-jar-with-dependenci…

Done
terrier-python-helper 0.0.8 jar not found, downloading to /root/.pyterrier...


https://repo1.maven.org/maven2/org/terrier/terrier-python-helper/0.0.8/terrier-python-helper-0.0.8.jar:   0%| …

Done


Java started (triggered by TerrierIndexer.__init__) and loaded: pyterrier.java.colab, pyterrier.java, pyterrier.java.24, pyterrier.terrier.java [version=5.11 (build: craig.macdonald 2025-01-13 21:29), helper_version=0.0.8]


beir/fiqa/test documents:   0%|          | 0/57638 [00:00<?, ?it/s]

19:50:30.535 [ForkJoinPool-1-worker-1] WARN org.terrier.structures.indexing.Indexer -- Adding an empty document to the index (7915) - further warnings are suppressed
19:50:42.275 [ForkJoinPool-1-worker-1] WARN org.terrier.structures.indexing.Indexer -- Indexed 39 empty documents


In [ ]:
print("Index folder:", 'beir-index')
print("Number of documents:", stats.getNumberOfDocuments())
print("Number of postings:", stats.getNumberOfPostings())
print("Number of tokens:", stats.getNumberOfTokens())
print("Number of unique terms:", stats.getNumberOfUniqueTerms())
print(f"Average document length:{stats.getAverageDocumentLength():.2f}")


Index folder: beir-index
Number of documents: 57638
Number of postings: 2714611
Number of tokens: 3783214
Number of unique terms: 51260
Average document length:65.64


# Baseline models

In [ ]:
bm25 = pt.terrier.Retriever(indexref, wmodel='BM25', controls={"qtype": "matchop"})
tfidf = pt.terrier.Retriever(indexref, wmodel="TF_IDF", controls={"qtype": "matchop"})

In [ ]:
queries['query'] = queries['query'].apply(rmv_problems)

In [ ]:
res_bm25 = bm25.transform(queries[['qid','query']])
res_tfidf = tfidf.transform(queries[['qid','query']])

In [ ]:
metrics = [P@1, P@5, P@10, R@5, R@10, nDCG@5, nDCG@10, AP]

1st baseline experiment

In [ ]:

pt.Experiment(
    [tfidf, bm25],
    queries,
    qrels,
    eval_metrics=metrics,
    names=["TF_IDF", "BM25"],
    #baseline=0,
    verbose=True,
)

pt.Experiment:   0%|          | 0/2 [00:00<?, ?system/s]

,name,AP,P@1,P@5,P@10,R@5,R@10,nDCG@5,nDCG@10
0,TF_IDF,0.209962,0.236111,0.108642,0.071142,0.249805,0.313278,0.231547,0.253659
1,BM25,0.210385,0.236111,0.106481,0.070370,0.247471,0.309708,0.230060,0.252589


2nd baseline experiment: custom formulas + RRF

In [ ]:
!pip install pyterrier_alpha

In [ ]:
import pyterrier_alpha as pta
combined = bm25 + tfidf
dph = pt.terrier.Retriever(indexref, wmodel="DPH")

rrf = pta.RRFusion(bm25, tfidf, k=60, num_results=1000)
weighted_linear = bm25 + 2 * dph

In [ ]:
systems = [bm25, tfidf, combined, weighted_linear, rrf]
names   = ["BM25", "TF_IDF", "(BM25+TFIDF)", "Weighted(BM25+2*TFIDF)", "RRF"]

results = pt.Experiment(
    systems,
    queries,
    qrels,
    metrics,
    names,
    perquery=False
)
display(results)

,name,AP,P@1,P@5,P@10,R@5,R@10,nDCG@5,nDCG@10
0,BM25,0.210385,0.236111,0.106481,0.070370,0.247471,0.309708,0.230060,0.252589
1,TF_IDF,0.209962,0.236111,0.108642,0.071142,0.249805,0.313278,0.231547,0.253659
2,(BM25+TFIDF),0.210414,0.236111,0.108025,0.070216,0.249713,0.308250,0.231324,0.252182
3,Weighted(BM25+2*TFIDF),0.208011,0.239198,0.107099,0.070216,0.242903,0.311580,0.228979,0.252283
4,RRF,0.209778,0.236111,0.108025,0.069907,0.248620,0.307249,0.230670,0.251379


3rd baseline experiment

In [ ]:
!pip install transformers

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

model_id = "meta-llama/Meta-Llama-3-8B-Instruct"

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    torch_dtype="auto"
)

tokenizer = AutoTokenizer.from_pretrained(model_id)

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer
)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


OSError: You are trying to access a gated repo.
Make sure to have access to it at https://huggingface.co/meta-llama/Meta-Llama-3-8B-Instruct.
401 Client Error. (Request ID: Root=1-6963ffe7-7baa1fca121d40ba4ff27b28;4829c991-d186-4ebc-a4e6-e34f98572fe6)

Cannot access gated repo for url https://huggingface.co/meta-llama/Meta-Llama-3-8B-Instruct/resolve/main/config.json.
Access to model meta-llama/Meta-Llama-3-8B-Instruct is restricted. You must have access to it and be authenticated to access it. Please log in.

In [ ]:
import re
import pyterrier as pt

def llm_expand_query(q: str, n_terms=10) -> str:
    prompt = (f'''
        Expand this search query for financial documents. \n"
        "Rules: Use synonyms, accounting terms, regulatory terms, filing/report types, and common abbreviations.
        Do not introduce new proper nouns (company names), tickers, years, currencies, or numbers.
        return comma-separated phrases only.\n'''
        f"Query: {q}\n"
        "Expansion:"
    )
    out = pipe(prompt, max_new_tokens=100, do_sample=False, return_full_text=False)[0]["generated_text"]

    terms = [t.strip() for t in re.split(r",|\n|;", out) if t.strip()]
    terms = terms[:n_terms]

    return q + " " + " ".join(terms)

llm_expander = pt.apply.generic(lambda df: df.assign(query=df["query"].map(llm_expand_query)))

bm25_llm = llm_expander >> bm25


In [ ]:
pt.Experiment(
    [bm25, bm25_llm],
    queries,
    qrels,
    eval_metrics=metrics,
    names=["BM25", "BM25 + LLM expansion"],
    verbose=True
)


# Advanced models

### Sentiment Analysis

In [ ]:
from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification

pipe = pipeline("text-classification", model="ProsusAI/finbert")

tokenizer_finbert = AutoTokenizer.from_pretrained("ProsusAI/finbert")
model_finbert = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert")

config.json:   0%|          | 0.00/758 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/252 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Device set to use cuda:0


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_finbert.to(device).eval()

LABELS = ["negative", "neutral", "positive"]

def iter_batches(corpus_iter, batch_size=32):
    batch = []
    for item in corpus_iter:
        # expects dict with keys: 'docno', 'text'
        batch.append(item)
        if len(batch) == batch_size:
            yield batch
            batch = []
    if batch:
        yield batch

rows = []
batch_size = 32  # increase (e.g., 64/128) if you have GPU memory

for batch in iter_batches(dataset.get_corpus_iter(), batch_size=batch_size):
    docnos = [d["docno"] for d in batch]
    texts  = [d["text"]  for d in batch]

    inputs = tokenizer_finbert(
        texts,
        return_tensors="pt",
        truncation=True,
        padding=True,
        max_length=512
    ).to(device)

    with torch.no_grad():
        logits = model_finbert(**inputs).logits

    probs = torch.softmax(logits, dim=-1)                 # (B, 3)
    pred_ids = probs.argmax(dim=-1).tolist()              # list length B
    confs = probs.max(dim=-1).values.tolist()             # confidence of chosen class

    for docno, pred_id, conf in zip(docnos, pred_ids, confs):
        rows.append({
            "docno": docno,
            "sentiment": LABELS[pred_id],
            "confidence": conf
        })

sentiment_df = pd.DataFrame(rows)
sentiment_df


beir/fiqa/test documents:   0%|          | 0/57638 [00:00<?, ?it/s]

,docno,sentiment,confidence
0,3,positive,0.740279
1,31,positive,0.911631
2,56,positive,0.881663
3,59,positive,0.922108
4,63,positive,0.936929
...,...,...,...
57633,599946,positive,0.875183
57634,599953,positive,0.859306
57635,599966,neutral,0.831728
57636,599975,positive,0.865308


In [ ]:
rows_q = []
for batch in iter_batches(queries.to_dict(orient='records'), batch_size=batch_size):
    qids = [d["qid"] for d in batch]
    qs  = [d["query"]  for d in batch]

    inputs = tokenizer_finbert(
        qs,
        return_tensors="pt",
        truncation=True,
        padding=True,
        max_length=512
    ).to(device)

    with torch.no_grad():
        logits = model_finbert(**inputs).logits

    probs = torch.softmax(logits, dim=-1)                 # (B, 3)
    pred_ids = probs.argmax(dim=-1).tolist()              # list length B
    confs = probs.max(dim=-1).values.tolist()             # confidence of chosen class

    for qid, pred_id, conf in zip(qids, pred_ids, confs):
        rows_q.append({
            "qid": qid,
            "sentiment": LABELS[pred_id],
            "confidence": conf
        })

sentiment_q = pd.DataFrame(rows_q)
sentiment_q


,qid,sentiment,confidence
0,4641,positive,0.771040
1,5503,positive,0.924082
2,7803,positive,0.937387
3,7017,positive,0.941313
4,10152,positive,0.860820
...,...,...,...
643,4102,positive,0.863959
644,3566,positive,0.933026
645,94,positive,0.916428
646,2551,positive,0.825843


### Named Entity Recognition

In [ ]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline

tokenizerNER = AutoTokenizer.from_pretrained("dslim/bert-base-NER")
modelNER = AutoModelForTokenClassification.from_pretrained("dslim/bert-base-NER")

nlp = pipeline("ner", model=modelNER, tokenizer=tokenizerNER)
example = "My name is Wolfgang and I live in Berlin"

ner_results = nlp(example)
print(ner_results)

tokenizer_config.json:   0%|          | 0.00/59.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/829 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/433M [00:00<?, ?B/s]

Some weights of the model checkpoint at dslim/bert-base-NER were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cuda:0


[{'entity': 'B-PER', 'score': np.float32(0.9990139), 'index': 4, 'word': 'Wolfgang', 'start': 11, 'end': 19}, {'entity': 'B-LOC', 'score': np.float32(0.999645), 'index': 9, 'word': 'Berlin', 'start': 34, 'end': 40}]


In [ ]:
# aggregation_strategy merges wordpieces and groups spans nicely
nlp = pipeline(
    "ner",
    model=modelNER,
    tokenizer=tokenizerNER,
    aggregation_strategy="simple"   # key improvement vs raw token outputs
)

rows_ner = []
batch_size_ner = 16  # increase if you have GPU / enough RAM

for batch in iter_batches(dataset.get_corpus_iter(), batch_size=batch_size_ner):
    docnos = [d["docno"] for d in batch]
    texts  = [d["text"] for d in batch]

    # pipeline supports list input => batched inference
    outputs = nlp(texts)

    # outputs is a list (per doc) of entity dicts
    for docno, ents in zip(docnos, outputs):
        words = [e.get("word", "") for e in ents if e.get("word")]
        rows_ner.append({"docno": docno, "entities": words})

df_ner = pd.DataFrame(rows_ner)
df_ner


Device set to use cuda:0


beir/fiqa/test documents:   0%|          | 0/57638 [00:00<?, ?it/s]

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


,docno,entities
0,3,[U. S]
1,31,"[DFA, CDO]"
2,56,"[FSA, FSA, FSA, FSA]"
3,59,"[Samsung, O, Samsung, Samsung, Samsung]"
4,63,"[SEC, 501, Regulation D, Em, ##ee Re, ##ent In..."
...,...,...
57633,599946,[]
57634,599953,[US]
57635,599966,[]
57636,599975,"[CT, CT, T]"


In [ ]:
rows_ner_q = []
for batch in iter_batches(queries.to_dict(orient='records'), batch_size=batch_size_ner):
    qids = [d["qid"] for d in batch]
    qs  = [d["query"]  for d in batch]

    # pipeline supports list input => batched inference
    outputs = nlp(texts)

    # outputs is a list (per doc) of entity dicts
    for qid, ents in zip(qids, outputs):
        words = [e.get("word", "") for e in ents if e.get("word")]
        rows_ner_q.append({"qid": qid, "entities": words})

df_ner_q = pd.DataFrame(rows_ner_q)
df_ner_q

,qid,entities
0,4641,[]
1,5503,[]
2,7803,[US]
3,7017,[]
4,10152,"[CT, CT, T]"
...,...,...
241,10812,[]
242,3512,[US]
243,4102,[]
244,3566,"[CT, CT, T]"


In [ ]:
tags_classifier = pipeline("zero-shot-classification",
                      model="facebook/bart-large-mnli")

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Device set to use cuda:0


In [ ]:
sequence_test = "one day I will see the world"
candidate_labels_test = ['travel', 'cooking', 'dancing', 'exploration']
tags_classifier(sequence_test, candidate_labels_test, multi_label=True)

{'sequence': 'one day I will see the world',
 'labels': ['travel', 'exploration', 'dancing', 'cooking'],
 'scores': [0.9945111274719238,
  0.9383887648582458,
  0.005706191528588533,
  0.0018193251453340054]}

In [ ]:
mnli_topics = ['investing', 'taxes', 'banking', 'education', 'credit', 'regulation']
sequence_fin_test = "I'm not saying I don't like the idea of on-the-job training too, but you can't expect the company to do that. Training workers is not their job - they're building software. Perhaps educational systems in the U.S. (or their students) should worry a little about getting marketable skills in exchange for their massive investment in education, rather than getting out with thousands in student debt and then complaining that they aren't qualified to do anything."
tags_classifier(sequence_fin_test, mnli_topics, multi_label=True)

{'sequence': "I'm not saying I don't like the idea of on-the-job training too, but you can't expect the company to do that. Training workers is not their job - they're building software. Perhaps educational systems in the U.S. (or their students) should worry a little about getting marketable skills in exchange for their massive investment in education, rather than getting out with thousands in student debt and then complaining that they aren't qualified to do anything.",
 'labels': ['education',
  'investing',
  'credit',
  'regulation',
  'banking',
  'taxes'],
 'scores': [0.9491311311721802,
  0.808022677898407,
  0.3284561038017273,
  0.05801757797598839,
  0.018773408606648445,
  0.0029581154230982065]}

In [ ]:

rows_topics = []
batch_size_topics = 64
TOPK = 3

topics = mnli_topics  # predefined labels/classes

for batch in iter_batches(dataset.get_corpus_iter(), batch_size=batch_size_topics):
    if not batch:   # <-- important: skip empty batches
        continue

    docnos = [d.get("docno") for d in batch]
    texts  = [d.get("text") for d in batch]

    # Keep only non-empty texts for inference, but remember positions
    valid = [(i, t) for i, t in enumerate(texts) if isinstance(t, str) and t.strip()]
    if not valid:
        # whole batch has empty/missing texts
        for docno in docnos:
            rows_topics.append({"docno": docno, "top3_topics": [], "top3_scores": []})
        continue

    valid_idx, valid_texts = zip(*valid)

    outputs = tags_classifier(list(valid_texts), topics, multi_label=True)

    # HF pipeline returns dict for single input; normalize to list
    if isinstance(outputs, dict):
        outputs = [outputs]

    # Build a per-doc default result
    top_topics = {i: ([], []) for i in range(len(texts))}

    # Fill in results for valid docs
    for i, out in zip(valid_idx, outputs):
        labels = out.get("labels", [])
        scores = out.get("scores", [])
        top_topics[i] = (labels[:TOPK], scores[:TOPK])

    # Emit rows for the whole batch
    for i, docno in enumerate(docnos):
        labs, scs = top_topics[i]
        rows_topics.append({"docno": docno, "top3_topics": labs, "top3_scores": scs})

df_topics = pd.DataFrame(rows_topics)
df_topics

beir/fiqa/test documents:   0%|          | 0/57638 [00:00<?, ?it/s]

,docno,top3_topics,top3_scores
0,3,"[education, investing, credit]","[0.9491311311721802, 0.808022677898407, 0.3284..."
1,31,"[credit, regulation, banking]","[0.9008433818817139, 0.6907130479812622, 0.665..."
2,56,"[regulation, credit, investing]","[0.9034470915794373, 0.5898595452308655, 0.173..."
3,59,"[credit, investing, regulation]","[0.9901934862136841, 0.595706582069397, 0.2746..."
4,63,"[regulation, investing, banking]","[0.7835092544555664, 0.4374961853027344, 0.216..."
...,...,...,...
57633,599946,"[regulation, taxes, credit]","[0.8661596179008484, 0.36633968353271484, 0.26..."
57634,599953,"[credit, taxes, regulation]","[0.5913487076759338, 0.5441805720329285, 0.386..."
57635,599966,"[regulation, credit, investing]","[0.3586137592792511, 0.3065590560436249, 0.273..."
57636,599975,"[education, regulation, credit]","[0.3447442948818207, 0.18888334929943085, 0.14..."


In [33]:
df_topics.to_csv('topics_df.csv')

In [36]:
rows_topics_q = []
batch_size_topics_q = 64
TOPK = 3

for batch in iter_batches(queries.to_dict(orient='records'), batch_size=batch_size_topics):
    if not batch:
        continue

    qids = [d["qid"] for d in batch]
    qs  = [d["query"]  for d in batch]

    valid = [(i, t) for i, t in enumerate(qs) if isinstance(t, str) and t.strip()]
    if not valid:
        for docno in qids:
            rows_topics.append({"docno": docno, "top3_topics": [], "top3_scores": []})
        continue

    valid_idx, valid_qs = zip(*valid)

    outputs = tags_classifier(list(valid_qs), topics, multi_label=True)

    if isinstance(outputs, dict):
        outputs = [outputs]

    top_topics = {i: ([], []) for i in range(len(qs))}

    for i, out in zip(valid_idx, outputs):
        labels = out.get("labels", [])
        scores = out.get("scores", [])
        top_topics[i] = (labels[:TOPK], scores[:TOPK])

    for i, qid in enumerate(qids):
        labs, scs = top_topics[i]
        rows_topics_q.append({"qids": qid, "top3_topics": labs, "top3_scores": scs})

df_topics_q = pd.DataFrame(rows_topics_q)
df_topics_q

,qids,top3_topics,top3_scores
0,4641,"[investing, banking, credit]","[0.4287841320037842, 0.3427979648113251, 0.056..."
1,5503,"[taxes, regulation, investing]","[0.9585870504379272, 0.04324774071574211, 0.01..."
2,7803,"[investing, banking, credit]","[0.8117352724075317, 0.3381434679031372, 0.306..."
3,7017,"[investing, credit, banking]","[0.5106886029243469, 0.2827812731266022, 0.083..."
4,10152,"[investing, banking, credit]","[0.461930513381958, 0.3720417022705078, 0.0511..."
...,...,...,...
643,4102,"[investing, banking, credit]","[0.9342626929283142, 0.4654264748096466, 0.365..."
644,3566,"[investing, credit, banking]","[0.8451239466667175, 0.06888141483068466, 0.02..."
645,94,"[credit, taxes, regulation]","[0.9780347347259521, 0.12368687987327576, 0.01..."
646,2551,"[investing, credit, regulation]","[0.0919739380478859, 0.0036584404297173023, 0...."


In [37]:
df_topics_q.to_csv('topics_df_q.csv')